In [1]:
## EDIT:
base_dir = '/glade/work/kjmayer/research/catalyst/TransferLearning/runmean_analysis/artificial_bias/perfectmodel_TLtest/E3SM_analysis/'

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import random
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time

import sys
sys.path.append(base_dir+'functions/')
from utils import split_retrain
from exp_hp import get_hp

2024-08-09 15:32:48.582450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-09 15:32:48.648539: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def get_gradients(inputs, top_pred_idx=None):
    """Computes the gradients of outputs w.r.t input image.

    Args:
        inputs: 2D/3D/4D matrix of samples
        top_pred_idx: (optional) Predicted label for the x_data
                      if classification problem. If regression,
                      do not include.

    Returns:
        Gradients of the predictions w.r.t img_input
    """
    inputs = tf.cast(inputs, tf.float32)

    with tf.GradientTape() as tape:
        tape.watch(inputs)
        
        # Run the forward pass of the layer and record operations
        # on GradientTape.
        preds = model(inputs, training=False)  
        
        # For classification, grab the top class
        if top_pred_idx is not None:
            preds = preds[:, top_pred_idx]
        
    # Use the gradient tape to automatically retrieve
    # the gradients of the trainable variables with respect to the loss.        
    grads = tape.gradient(preds, inputs)
    return grads

def get_integrated_gradients(inputs, baseline=None, num_steps=50, top_pred_idx=None):
    """Computes Integrated Gradients for a prediction.

    Args:
        inputs (ndarray): 2D/3D/4D matrix of samples
        baseline (ndarray): The baseline image to start with for interpolation
        num_steps: Number of interpolation steps between the baseline
            and the input used in the computation of integrated gradients. These
            steps along determine the integral approximation error. By default,
            num_steps is set to 50.
        top_pred_idx: (optional) Predicted label for the x_data
                      if classification problem. If regression,
                      do not include.            

    Returns:
        Integrated gradients w.r.t input image
    """
    # If baseline is not provided, start with zeros
    # having same size as the input image.
    if baseline is None:
        input_size = np.shape(inputs)[1:]
        baseline = np.zeros(input_size).astype(np.float32)
    else:
        baseline = baseline.astype(np.float32)

    # 1. Do interpolation.
    inputs = inputs.astype(np.float32)
    interpolated_inputs = [
        baseline + (step / num_steps) * (inputs - baseline)
        for step in range(num_steps + 1)
    ]
    interpolated_inputs = np.array(interpolated_inputs).astype(np.float32)

    # 3. Get the gradients
    grads = []
    for i, x_data in enumerate(interpolated_inputs):
        grad = get_gradients(x_data, top_pred_idx=top_pred_idx)     
        grads.append(grad)
    grads = tf.convert_to_tensor(grads, dtype=tf.float32)

    # 4. Approximate the integral using the trapezoidal rule
    grads = (grads[:-1] + grads[1:]) / 2.0
    avg_grads = tf.reduce_mean(grads, axis=0)
    # 5. Calculate integrated gradients and return
    integrated_grads = (inputs - baseline) * avg_grads
    return integrated_grads

In [3]:
#----- SET UP & TRAIN NN -----
EXP_NAME = 'exp2'
EXP_NAME2 = 'exp2_retrain_increase'
hps2 = get_hp(EXP_NAME2)

GLOBAL_SEED = hps2['GLOBAL_SEED']
np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
tf.random.set_seed(GLOBAL_SEED)

print('Validation Member: '+str(hps2['val_mems'][0]))
print('Testing Member: '+str(hps2['test_mems'][0]))

for i_trainmems in np.arange(3,9):
    print(i_trainmems)
    print('Training Members: '+str(hps2['train_mems'][i_trainmems]))

    _, _, X1test, _, _, Y1test,_ = split_retrain(trainmems = hps2['train_mems'][i_trainmems],
                                                  valmem = hps2['val_mems'][0],
                                                  testmem = hps2['test_mems'][0],
                                                  months = [11,12,1,2], # months for X (Y+leadtime are accounted for in function)
                                                  lead = hps2['LEAD'])

    nlat = X1test.shape[1]
    nlon = X1test.shape[2]
    X1test = X1test.reshape((X1test.shape[0],nlat*nlon))
    number_inputs = nlat*nlon
    
    for SEED in np.arange(0,10):
        IG_neg_heatmap = []
        IG_pos_heatmap = []
        print(SEED)
        annfi_name = 'ann2_60Eshift_'+EXP_NAME2+'.'+str(i_trainmems)+'_ann1-'+EXP_NAME+'_seed'+str(SEED)+'.h5'
        model = tf.keras.models.load_model(base_dir+'train/saved_models/'+annfi_name)
        
        pred = model.predict(X1test)
        predconf = np.max(pred,axis=-1)
        predval  = np.argmax(pred,axis=-1)
    
        iconf = np.where(predconf >= np.percentile(predconf,q=80))
        icorr = np.where(predval[iconf] == Y1test[iconf])
        
        ineg = np.where(Y1test[iconf][icorr] == 0)
        ipos = np.where(Y1test[iconf][icorr] == 1)
        
        # print(len(ineg[0]))
        # print(len(ipos[0]))
        
        if len(ineg[0]) > 0:
            for neg_sample in X1test[iconf][icorr][ineg]:
                IG_negsample_heatmap = get_integrated_gradients(neg_sample.reshape((1,number_inputs)),top_pred_idx=0).numpy()[0]
                IG_neg_heatmap.append(IG_negsample_heatmap/np.max(np.abs(IG_negsample_heatmap)))
            
            IG_neg_heatmap_reshape = np.asarray(IG_neg_heatmap).reshape((len(ineg[0]),nlat,nlon))
            np.save(base_dir+'IG/data/IG_ann2_60Eshift_'+EXP_NAME2+'.'+str(i_trainmems)+'_ann1-'+EXP_NAME+'_negconf_seed'+str(SEED)+'.npy',
                    IG_neg_heatmap_reshape,
                    allow_pickle=True)
        
        if len(ipos[0]) > 0:
            for pos_sample in X1test[iconf][icorr][ipos]:
                IG_possample_heatmap = get_integrated_gradients(pos_sample.reshape((1,number_inputs)),top_pred_idx=1).numpy()[0]
                IG_pos_heatmap.append(IG_possample_heatmap/np.max(np.abs(IG_possample_heatmap)))
                
            IG_pos_heatmap_reshape = np.asarray(IG_pos_heatmap).reshape((len(ipos[0]),nlat,nlon))
            np.save(base_dir+'IG/data/IG_ann2_60Eshift_'+EXP_NAME2+'.'+str(i_trainmems)+'_ann1-'+EXP_NAME+'_posconf_seed'+str(SEED)+'.npy',
                    IG_pos_heatmap_reshape,
                    allow_pickle=True)

Validation Member: 0291
Testing Member: 0301
3
Training Members: ['0201', '0211', '0221', '0231']
loading data & saving
0


2024-08-09 15:33:55.544635: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


242/242 [==============================] - 1s 2ms/step
1
242/242 [==============================] - 1s 2ms/step
2
242/242 [==============================] - 0s 1ms/step
3
242/242 [==============================] - 0s 1ms/step
4
242/242 [==============================] - 0s 1ms/step
5
242/242 [==============================] - 0s 809us/step
6
242/242 [==============================] - 0s 762us/step
7
242/242 [==============================] - 0s 897us/step
8
242/242 [==============================] - 0s 801us/step
9
242/242 [==============================] - 0s 2ms/step
4
Training Members: ['0201', '0211', '0221', '0231', '0241']
loading data & saving
0
239/239 [==============================] - 0s 1ms/step
1
239/239 [==============================] - 0s 815us/step
2
239/239 [==============================] - 0s 1ms/step
3
239/239 [==============================] - 0s 1ms/step
4
239/239 [==============================] - 0s 767us/step
5
239/239 [==============================] - 0s 1ms/

In [7]:
# IG_neg_heatmap_mean = np.mean(np.asarray(IG_neg_heatmap).reshape((len(ineg[0]),nlat,nlon)),0)
# IG_pos_heatmap_mean = np.mean(IG_pos_heatmap_reshape,0)#np.asarray(IG_pos_heatmap).reshape((len(ipos[0]),nlat,nlon)),0)

In [8]:
# plt.imshow(IG_neg_heatmap_mean,origin='lower',cmap='RdBu_r',vmin=-.1,vmax=.1)
# plt.title('Unbiased Testing Member - After TL')
# plt.show()
# plt.imshow(IG_pos_heatmap_mean,origin='lower',cmap='RdBu_r',vmin=-.1,vmax=.1)
# plt.show()